In [1]:
import pandas as pd
import mysql.connector
import os

In [9]:
path = './文件集/補校區文件/'
dirList = os.listdir(path)
dfSchool = pd.read_excel("./文件集/學校統整v4.xlsx")

## Version1

In [2]:
dfSchool = pd.read_excel("./文件集/學校統整V3.xlsx")
dfSchool = dfSchool.astype('str')

### 建學校database資料

In [53]:
data = []
for schoolID in dfSchool['學校ID'].unique():
    mask = dfSchool['學校ID'] == schoolID
    data.append(tuple(dfSchool[mask].iloc[0][['學校ID', '校名','網址']]))

In [73]:
db = mysql.connector.connect(user='developer1', password='password', host='localhost', database='四技二專')
cursor = db.cursor()
sql = 'insert into 學校 values(%s, %s, %s)'
cursor.executemany(sql, data)
db.commit()
cursor.close()
db.close()

### 建分校database資料

In [3]:
def checkValid(campusInfo):
    if(len(campusInfo) == 1):
        if(campusInfo[0][0] == "0" and campusInfo[0][1] != "0"):
            return True
        else:
            return False
    else:
        for i in range(len(campusInfo)):
            if(campusInfo[i][0] == "0" or campusInfo[i][1] == '0'):
                return False
        return True
    
def findAlias(string):
    alias = ''
    for char in string:
        if(char == "台"):
            alias += '臺'
        elif(char == '臺'):
            alias += '台'
        else:
            alias += char
    return alias

In [4]:
dicCampusInSch = {}
for schoolID in dfSchool['學校ID'].unique():
    dicCampusInSch[schoolID] = []
    mask = dfSchool['學校ID'] == schoolID
    dfTemp = dfSchool[mask]
    for campus in dfTemp['校區'].unique():
        maskCampus = dfTemp['校區'] == campus
        seriInfo = dfTemp[maskCampus].iloc[0]
        dicCampusInSch[schoolID].append([campus, seriInfo['地址'], seriInfo['縣市'], seriInfo['行政區']])
dicCampusInSch

{'101': [['0', '106台北市大安區基隆路四段43號', '臺北市', '大安區']],
 '102': [['0', '64002雲林縣斗六市大學路三段123號', '雲林縣', '斗六市']],
 '103': [['0', '912屏東縣內埔鄉學府路1號', '屏東縣', '內埔鄉']],
 '104': [['0', '10608台北市大安區忠孝東路三段1號', '臺北市', '大安區']],
 '105': [['0', '0', '高雄市', '三民區'], ['建工校區', '0', '高雄市', '三民區']],
 '107': [['0', '632雲林縣虎尾鎮文化路64號', '雲林縣', '虎尾鎮']],
 '109': [['0', '880澎湖縣馬公市六合路300號', '澎湖縣', '馬公市']],
 '110': [['0', '411030台中市太平區中山路二段57號', '臺中市', '太平區']],
 '111': [['0', '0', '臺北市', '北投區']],
 '112': [['0', '812高雄市小港區松和路1號', '高雄市', '小港區']],
 '113': [['0', '404台中市北區三民路三段129號', '臺中市', '北區']],
 '114': [['桃園校區', '324桃園市平鎮區福龍路一段100號', '臺北市', '中正區'],
  ['臺北校區', '100台北市中正區濟南路一段321號', '臺北市', '中正區']],
 '201': [['0', '413台中市霧峰區吉峰東路168號', '臺中市', '霧峰區']],
 '202': [['0', '71005台南市永康區南台街1號', '臺南市', '永康區']],
 '203': [['0', '710台南市永康區崑大路195號', '臺南市', '永康區']],
 '204': [['0', '71710 臺南市仁德區二仁路一段60號', '臺南市', '仁德區']],
 '205': [['0', '824高雄市燕巢區橫山路59號', '高雄市', '燕巢區']],
 '206': [['0', '333桃園市龜山區萬壽路一段300號', '桃園市', '龜山區']],
 '207': [['0', '8

In [6]:
db = mysql.connector.connect(user='root', password='root', host='localhost', database='四技二專')
cursor = db.cursor()
sql = """SELECT t2.地區ID, t1.縣市ID, t1.行政區ID, t2.縣市名稱, t1.行政區名稱 
FROM 鄉鎮市區 as t1 LEFT JOIN 縣市 AS t2 ON t1.縣市ID = t2.縣市ID
WHERE (t2.縣市名稱 = %s OR t2.縣市名稱 = %s) AND
(t1.行政區名稱 = %s OR t1.行政區名稱 = %s) ;"""

data = []
for key in dicCampusInSch.keys():
    campusInfo = dicCampusInSch[key]
    if(checkValid(campusInfo)):
        for i in range(len(campusInfo)):
            cityAlias, zoneAlias = findAlias(campusInfo[i][2]), findAlias(campusInfo[i][3])
            cursor.execute(sql, (campusInfo[i][2], cityAlias, campusInfo[i][3], zoneAlias))
            row = cursor.fetchall()[0]
            data.append((None, campusInfo[i][0], campusInfo[i][1], key, row[0], row[1], row[2]))

In [7]:
data

[(None, '0', '106台北市大安區基隆路四段43號', '101', 1, 1, 5),
 (None, '0', '64002雲林縣斗六市大學路三段123號', '102', 2, 12, 185),
 (None, '0', '912屏東縣內埔鄉學府路1號', '103', 3, 17, 302),
 (None, '0', '10608台北市大安區忠孝東路三段1號', '104', 1, 1, 5),
 (None, '0', '632雲林縣虎尾鎮文化路64號', '107', 2, 12, 178),
 (None, '0', '880澎湖縣馬公市六合路300號', '109', 3, 18, 324),
 (None, '0', '411030台中市太平區中山路二段57號', '110', 2, 9, 116),
 (None, '0', '812高雄市小港區松和路1號', '112', 3, 16, 262),
 (None, '0', '404台中市北區三民路三段129號', '113', 2, 9, 112),
 (None, '桃園校區', '324桃園市平鎮區福龍路一段100號', '114', 1, 1, 1),
 (None, '臺北校區', '100台北市中正區濟南路一段321號', '114', 1, 1, 1),
 (None, '0', '413台中市霧峰區吉峰東路168號', '201', 2, 9, 118),
 (None, '0', '71005台南市永康區南台街1號', '202', 3, 15, 222),
 (None, '0', '710台南市永康區崑大路195號', '203', 3, 15, 222),
 (None, '0', '71710 臺南市仁德區二仁路一段60號', '204', 3, 15, 229),
 (None, '0', '824高雄市燕巢區橫山路59號', '205', 3, 16, 270),
 (None, '0', '333桃園市龜山區萬壽路一段300號', '206', 1, 4, 56),
 (None, '0', '831高雄市大寮區進學路151號', '207', 3, 16, 277),
 (None, '0', '304新竹縣新豐鄉新興路1號', '208', 1

In [8]:
sql = "insert into 分校 values(%s, %s, %s, %s, %s, %s, %s);"
cursor.executemany(sql, data)
db.commit()

In [9]:
cursor.close()
db.close()